## Handling Approach (99000 vs 1000)
### 1. take 1000 samples from 99000 and discard the rest
### 2. over sampling minority class by duplication (1000 x 99)
### 3. (BEST METHOD) generate synthetic examples using k-nearest neighbors algorithm (SMOTE: Synthetic Minority Over-sampling technique)
### 4. divide the larger sample into multiple samples and train each of them, and then take a majority vote (ensemble method)
### 5. penalize majority samples duing loss calculation and give more weight to minority class (Focal Loss)

### Examples: churn rate, device failure, cancer prediciton

In [18]:
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline

# import and preprocessing
df = pd.read_csv("9_Customer_churn.csv")
df.sample(5)
df.drop("customerID", axis="columns", inplace = True)
pd.to_numeric(df.TotalCharges, errors="coerce")
df[pd.to_numeric(df.TotalCharges, errors="coerce").isnull()]
df1 = df[df.TotalCharges != " "]
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)
df1.replace("No internet service", "No", inplace=True)
df1.replace("No phone service", "No", inplace=True)
df1["gender"].replace({"Female": 1, "Male": 0}, inplace=True)
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_columns:
    df1[col].replace({'Yes': 1,'No': 0},inplace=True)
df2 = pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']
scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [19]:
# imbalanced dataset
count_0, count_1 = df1.Churn.value_counts()
df_class_0 = df2[df2["Churn"] == 0]
df_class_1 = df2[df2["Churn"] == 1]

df_class_0.shape, df_class_1.shape

((5163, 27), (1869, 27))

## Method 1: Undersampling

In [20]:
df_class_0_under = df_class_0.sample(count_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)
df_test_under.Churn.value_counts()

0    1869
1    1869
Name: Churn, dtype: int64

In [22]:
X = df_test_under.drop("Churn", axis="columns")
y = df_test_under["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts() # y train has equal sample size because of stratify

0    1495
1    1495
Name: Churn, dtype: int64

In [24]:
# model
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(26, input_dim=26, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return y_preds

y_pred = ANN(X_train, y_train, X_test, y_test, "binary_crossentropy", -1)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.5725 - accuracy: 0.7120
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4964 - accuracy: 0.7615
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4885 - accuracy: 0.7652
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4825 - accuracy: 0.7659
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4801 - accuracy: 0.7682
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4764 - accuracy: 0.7736
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4760 - accuracy: 0.7739
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4730 - accuracy: 0.7729
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4741 - accuracy: 0.7726
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4692 - accuracy: 0.7763
Epoch 11/

## Method 2: Oversampling

In [28]:
df_class_1_over = df_class_1.sample(count_0, replace=True) # duplicate samples
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

In [29]:
X = df_test_over.drop("Churn", axis="columns")
y = df_test_over["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts() # y train has equal sample size because of stratify

0    4130
1    4130
Name: Churn, dtype: int64

In [30]:
y_pred = ANN(X_train, y_train, X_test, y_test, "binary_crossentropy", -1)
# better than 1

Epoch 1/100
259/259 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7217
Epoch 2/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4969 - accuracy: 0.7580
Epoch 3/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4876 - accuracy: 0.7656
Epoch 4/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4816 - accuracy: 0.7649
Epoch 5/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4764 - accuracy: 0.7677
Epoch 6/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4726 - accuracy: 0.7749
Epoch 7/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4701 - accuracy: 0.7706
Epoch 8/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.7734
Epoch 9/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.7720
Epoch 10/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4614 - accura

## Method 3: SMOTE

In [39]:
from imblearn.over_sampling import SMOTE
X = df2.drop("Churn", axis="columns")
y = df2["Churn"]

smote = SMOTE(sampling_strategy="minority")
X, y = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [40]:
y_pred = ANN(X_train, y_train, X_test, y_test, "binary_crossentropy", -1)

Epoch 1/100
259/259 [==============================] - 1s 2ms/step - loss: 0.5219 - accuracy: 0.7458
Epoch 2/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4719 - accuracy: 0.7736
Epoch 3/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4638 - accuracy: 0.7774
Epoch 4/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4564 - accuracy: 0.7814
Epoch 5/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4504 - accuracy: 0.7855
Epoch 6/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4440 - accuracy: 0.7889
Epoch 7/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4391 - accuracy: 0.7941
Epoch 8/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4347 - accuracy: 0.7977
Epoch 9/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.8002
Epoch 10/100
259/259 [==============================] - 0s 2ms/step - loss: 0.4274 - accura

## Method 4: Ensemble | undersampling

In [41]:
X = df2.drop("Churn", axis="columns")
y = df2["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [42]:
4130 / 1495 # divide to approx 3 batches

2.762541806020067

In [59]:
df3 = X_train.copy()
df3['Churn'] = y_train
df3_class0 = df3[df3.Churn==0]
df3_class1 = df3[df3.Churn==1]

def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train

X_train, y_train = get_train_batch(df3_class0, df3_class1, 0, 1495)
y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

X_train, y_train = get_train_batch(df3_class0, df3_class1, 1495, 2990)
y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

X_train, y_train = get_train_batch(df3_class0, df3_class1, 2990, 4130)
y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

- accuracy: 1.0000
Epoch 36/100
47/47 [==============================] - 0s 2ms/step - loss: 3.5007e-06 - accuracy: 1.0000
Epoch 37/100
47/47 [==============================] - 0s 2ms/step - loss: 3.2762e-06 - accuracy: 1.0000
Epoch 38/100
47/47 [==============================] - 0s 2ms/step - loss: 3.0742e-06 - accuracy: 1.0000
Epoch 39/100
47/47 [==============================] - 0s 2ms/step - loss: 2.8904e-06 - accuracy: 1.0000
Epoch 40/100
47/47 [==============================] - 0s 2ms/step - loss: 2.7178e-06 - accuracy: 1.0000
Epoch 41/100
47/47 [==============================] - 0s 2ms/step - loss: 2.5626e-06 - accuracy: 1.0000
Epoch 42/100
47/47 [==============================] - 0s 2ms/step - loss: 2.4184e-06 - accuracy: 1.0000
Epoch 43/100
47/47 [==============================] - 0s 2ms/step - loss: 2.2864e-06 - accuracy: 1.0000
Epoch 44/100
47/47 [==============================] - 0s 2ms/step - loss: 2.1623e-06 - accuracy: 1.0000
Epoch 45/100
47/47 [=========================

In [60]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones>1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [63]:
print(classification_report(y_test, y_pred_final, zero_division=True))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1033
           1       0.27      1.00      0.42       374

    accuracy                           0.27      1407
   macro avg       0.63      0.50      0.21      1407
weighted avg       0.80      0.27      0.11      1407

